In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import SelectPercentile
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [28]:
df_train_origin = pd.read_csv('data/application_train.csv')
df_test_origin = pd.read_csv('data/application_test.csv')

combine_origin = [df_train_origin,df_test_origin]

In [154]:
# df_train[["FLAG_OWN_CAR", "TARGET"]].groupby(['FLAG_OWN_CAR'], as_index=False).mean().sort_values(by='TARGET', ascending=False)

# 收教育程度高低和逾期存在关系
# df_train_origin[["NAME_EDUCATION_TYPE", "TARGET"]].groupby(['NAME_EDUCATION_TYPE'], as_index=False).mean().sort_values(by='TARGET', ascending=False)

# 通过箱型图观察，每期还款金额会影响逾期
# df_train_origin[df_train_origin['TARGET'] == 1][['AMT_ANNUITY']].plot.box()
# df_train_origin[df_train_origin['TARGET'] == 0][['AMT_ANNUITY']].plot.box()



In [186]:
df_train_origin[df_train_origin['NAME_EDUCATION_TYPE'] == 'Secondary / secondary special'].shape

(218391, 122)

In [29]:
# combine_origin[0].head()

In [165]:
df_train_origin[df_train_origin['NAME_TYPE_SUITE'].isnull()].shape

(0, 122)

In [151]:
# TARGET：1-会逾期；2-其他情况
# NAME_CONTRACT_TYPE（合同类型-贷款方式）：Cash loans（现金贷款）；Revolving loans（循环贷款）
# CODE_GENDER（客户性别）：F（女）；M（男）；XNA（）
# FLAG_OWN_CAR（是否拥有车辆）：N/Y
# FLAG_OWN_REALTY（是否拥有房产）：N/Y
# CNT_CHILDREN（拥有子女数量）：int
# AMT_INCOME_TOTAL（收入）：
# AMT_CREDIT（贷款的信贷金额）：
# AMT_ANNUITY（年金-每期还款）
# AMT_GOODS_PRICE（商品价格）

# NAME_TYPE_SUITE（客户申请贷款时由谁陪同）：1.Unaccompanied（无陪同）；2.Family；3.Spouse, partner（配偶伴侣）；4.Children；5.Other_B；6.Other_A；7.Group of people
# NAME_INCOME_TYPE（收入类型）：1.Working；2.Commercial associate（商业伙伴）；3.Pensioner（养老金）；4.State servant（）；5.Unemployed（失业）；6.Student；7.Businessman；8.Maternity leave
# NAME_EDUCATION_TYPE（受教育程度）：1.Secondary / secondary special（高中）；2.Higher education（大学以上学历）；3.Incomplete higher（大学在读/未完成大学）；4.Lower secondary（初中）；5.Academic degree（学位）
# NAME_FAMILY_STATUS（家庭情况）：1.Married；2.Single / not married；3.Civil marriage；4.Separated；5.Widow；6.Unknown
# NAME_HOUSING_TYPE（住房情况）：1.House / apartment；2.With parents；3.Municipal apartment；4.Rented apartment；5.Office apartment；6.Co-op apartment
# REGION_POPULATION_RELATIVE（数量越高表示客户所在区域人口越多）：
# DAYS_BIRTH（年龄-天）：
# DAYS_EMPLOYED（当前的工作持续时间-天）：
# DAYS_LAST_PHONE_CHANGE（-天）：
# DAYS_REGISTRATION（客户在申请前几天更改了注册-天）：
# DAYS_ID_PUBLISH（客户在申请前多少天更改了申请贷款的身份证明文件-天）：
# OWN_CAR_AGE（车的年龄）：1.NaN；2.int
# FLAG_MOBIL（是否预留的手机号）：1/0
# FLAG_EMP_PHONE：1/0
# FLAG_WORK_PHONE：1/0
# FLAG_CONT_MOBILE：1/0
# FLAG_PHONE：1/0
# FLAG_EMAIL：1/0
# FLAG_DOCUMENT_2（是否提供了文件2~21）：1/0
# OCCUPATION_TYPE（职业）：1.Laborers；2.Sales staff；3.Core staff；4.Managers；5.Drivers；6.High skill tech staff；7.Accountants；8.Medicine staff；9.Security staff；10.Cooking staff；
#                          11.Cleaning staff；12.Private service staff；13.Low-skill Laborers；14.Waiters/barmen staff；15.Secretaries；16.Realty agents；17.HR staff；18.IT staff
# ORGANIZATION_TYPE（工作的组织类型）：类型比较多
# CNT_FAM_MEMBERS（家庭成员数量）：int
# REGION_RATING_CLIENT（我们对客户所在地区的评级）：1/2/3
# REGION_RATING_CLIENT_W_CITY（考虑城市因素，我们对客户所在地区的评级）：1/2/3
# WEEKDAY_APPR_PROCESS_START（申请贷款的在一周中的哪一天）：1.SUNDAY；2.SATURDAY；3.FRIDAY；4.THURSDAY；5.MONDAY；6.WEDNESDAY；7.TUESDAY
# HOUR_APPR_PROCESS_START（申请贷款的在一周中的几点）：0~23
# REG_REGION_NOT_LIVE_REGION（永久地址与联系地址是否在同一区域级别）：0（相同）；1（不同）
# REG_REGION_NOT_WORK_REGION（永久地址与工作地址是否在同一区域级别）：0（相同）；1（不同）
# LIVE_REGION_NOT_WORK_REGION（联系地址与工作地址是否在同一区域级别）：0（相同）；1（不同）
# REG_CITY_NOT_LIVE_CITY（永久地址与联系地址是否在同一城市级别）：0/1
# REG_CITY_NOT_WORK_CITY：0/1
# LIVE_CITY_NOT_WORK_CITY：0/1
# EXT_SOURCE_1（来自外部数据源的标准化评分）：0~1
# EXT_SOURCE_2
# EXT_SOURCE_3

## 客户居住的建筑的标准化信息，平均(_AVG后缀)，modus后缀(_MODE后缀)，中值(_MEDI后缀)  0~1 
## 公寓大小，公共面积，居住面积，建筑年龄，电梯数量，入口数量，建筑状态，楼层数
# APARTMENTS_AVG（公寓大小）：1.NaN；2.0~1
# BASEMENTAREA_AVG（地下室）：1.NaN；2.0~1
# YEARS_BEGINEXPLUATATION_AVG：1.NaN；2.0~1
# YEARS_BUILD_AVG：1.NaN；2.0~1
# COMMONAREA_AVG：1.NaN；2.0~1
# ELEVATORS_AVG：1.NaN；2.0~1
# ENTRANCES_AVG：1.NaN；2.0~1
# FLOORSMAX_AVG：1.NaN；2.0~1
# FLOORSMIN_AVG：1.NaN；2.0~1
# LANDAREA_AVG：1.NaN；2.0~1
# LIVINGAPARTMENTS_AVG：1.NaN；2.0~1
# LIVINGAREA_AVG：1.NaN；2.0~1
# NONLIVINGAPARTMENTS_AVG：1.NaN；2.0~1
# NONLIVINGAREA_AVG：1.NaN；2.0~1

# OBS_30_CNT_SOCIAL_CIRCLE（逾期30天，对客户社交环境的观察有多少次）：
# DEF_30_CNT_SOCIAL_CIRCLE（逾期30天的客户社会环境观察值）
# OBS_60_CNT_SOCIAL_CIRCLE
# DEF_60_CNT_SOCIAL_CIRCLE

# AMT_REQ_CREDIT_BUREAU_HOUR（在申请前一小时向信用局询问客户的数量）
# AMT_REQ_CREDIT_BUREAU_DAY（在申请前一天(不包括申请前一小时)向信贷局查询客户的数目）
# AMT_REQ_CREDIT_BUREAU_WEEK
# AMT_REQ_CREDIT_BUREAU_MON
# AMT_REQ_CREDIT_BUREAU_QRT
# AMT_REQ_CREDIT_BUREAU_YEAR

In [30]:
combine = []

for i,dataset in enumerate(combine_origin):
    # 由于字段比较多，先把需要的提取出来
    df = pd.DataFrame()
    try:
        df = dataset[['SK_ID_CURR','TARGET']]
    except:
        df = dataset[['SK_ID_CURR']]
    
#     income_median = dataset['AMT_INCOME_TOTAL'].median()
#     a = np.linspace(income_median * 0.5,income_median * 1.5,num = 4)
    
#     dataset.loc[dataset['AMT_INCOME_TOTAL'] < a[0],'AMT_INCOME_TOTAL'] = 1
#     dataset.loc[(dataset['AMT_INCOME_TOTAL'] >= a[0]) & (dataset['AMT_INCOME_TOTAL'] < a[1]),'AMT_INCOME_TOTAL'] = 2
#     dataset.loc[(dataset['AMT_INCOME_TOTAL'] >= a[1]) & (dataset['AMT_INCOME_TOTAL'] < a[2]),'AMT_INCOME_TOTAL'] = 3
#     dataset.loc[(dataset['AMT_INCOME_TOTAL'] >= a[2]) & (dataset['AMT_INCOME_TOTAL'] < a[3]),'AMT_INCOME_TOTAL'] = 4
#     dataset.loc[dataset['AMT_INCOME_TOTAL'] >= a[3],'AMT_INCOME_TOTAL'] = 5


    dataset.replace(np.nan, 0, inplace=True) 
    dataset.replace(np.inf, 0, inplace=True)
    
    dataset['AMT_INCOME_TOTAL'] = dataset['AMT_INCOME_TOTAL'].apply(lambda x:round(x / 10000,2))
    
    dataset['NAME_CONTRACT_TYPE'] = dataset['NAME_CONTRACT_TYPE'].map({"Cash loans":1,"Revolving loans":2})
    dataset['CODE_GENDER'] = dataset['CODE_GENDER'].map({"F":1,"M":2,"XNA":3})
    dataset['FLAG_OWN_CAR'] = dataset['FLAG_OWN_CAR'].map({"N":0,"Y":1})
    dataset['FLAG_OWN_REALTY'] = dataset['FLAG_OWN_REALTY'].map({"N":0,"Y":1})
    
#     dataset['NAME_TYPE_SUITE'] = dataset['NAME_TYPE_SUITE'].fillna(0)
    dataset['NAME_TYPE_SUITE'] = dataset['NAME_TYPE_SUITE'].map({"Unaccompanied":1,"Family":2,"Spouse, partner":3,"Children":4,"Other_B":5,"Other_A":6,"Group of people":7})
    dataset['NAME_INCOME_TYPE'] = dataset['NAME_INCOME_TYPE'].map({"Working":1,"Commercial associate":2,"Pensioner":3,"State servant":4,"Unemployed":5,"Student":6,"Businessman":7,"Maternity leave":8})
    dataset['NAME_FAMILY_STATUS'] = dataset['NAME_FAMILY_STATUS'].map({"Married":1,"Single / not married":2,"Civil marriage":3,"Separated":4,"Widow":5,"Unknown":6})
    dataset['NAME_HOUSING_TYPE'] = dataset['NAME_HOUSING_TYPE'].map({"House / apartment":1,"With parents":2,"Municipal apartment":3,"Rented apartment":4,"Office apartment":5,"Co-op apartment":6})
#     dataset.loc[dataset['OWN_CAR_AGE'].isnull(),'OWN_CAR_AGE'] = 0
#     dataset.loc[dataset['CNT_FAM_MEMBERS'].isnull(),'CNT_FAM_MEMBERS'] = 0
    #职业比较多，以后再分析
    dataset = dataset.drop(['OCCUPATION_TYPE'], axis=1)
    dataset = dataset.drop(['ORGANIZATION_TYPE'], axis=1)
    
    dataset['WEEKDAY_APPR_PROCESS_START'] = dataset['WEEKDAY_APPR_PROCESS_START'].map({"SUNDAY":1,"SATURDAY":2,"FRIDAY":3,"THURSDAY":4,"MONDAY":5,"WEDNESDAY":6,"TUESDAY":7})
    
    for suffix in ['_AVG','_MODE','_MEDI']:
        dataset = dataset.drop(['APARTMENTS' + suffix], axis=1)
        dataset = dataset.drop(['BASEMENTAREA' + suffix], axis=1)
        dataset = dataset.drop(['YEARS_BEGINEXPLUATATION' + suffix], axis=1)
        dataset = dataset.drop(['YEARS_BUILD' + suffix], axis=1)
        dataset = dataset.drop(['COMMONAREA' + suffix], axis=1)
        dataset = dataset.drop(['ELEVATORS' + suffix], axis=1)
        dataset = dataset.drop(['ENTRANCES' + suffix], axis=1)
        dataset = dataset.drop(['FLOORSMAX' + suffix], axis=1)
        dataset = dataset.drop(['FLOORSMIN' + suffix], axis=1)
        dataset = dataset.drop(['LANDAREA' + suffix], axis=1)
        dataset = dataset.drop(['LIVINGAPARTMENTS' + suffix], axis=1)
        dataset = dataset.drop(['LIVINGAREA' + suffix], axis=1)
        dataset = dataset.drop(['NONLIVINGAPARTMENTS' + suffix], axis=1)
        dataset = dataset.drop(['NONLIVINGAREA' + suffix], axis=1)
        
    dataset = dataset.drop(['FONDKAPREMONT_MODE'], axis=1)
    dataset = dataset.drop(['HOUSETYPE_MODE'], axis=1)
    dataset = dataset.drop(['TOTALAREA_MODE'], axis=1)
    dataset = dataset.drop(['WALLSMATERIAL_MODE'], axis=1)
    dataset = dataset.drop(['EMERGENCYSTATE_MODE'], axis=1)
#     for i in range(1,4):
#         dataset.loc[dataset['EXT_SOURCE_' + str(i)].isnull(),'EXT_SOURCE_' + str(i)] = 0
        
#     for i in range(2,22):
#         dataset.loc[dataset['FLAG_DOCUMENT_' + str(i)].isnull(),'FLAG_DOCUMENT_' + str(i)] = 0
    
#     dataset.loc[dataset['OBS_30_CNT_SOCIAL_CIRCLE' + suffix].isnull(),'OBS_30_CNT_SOCIAL_CIRCLE' + suffix] = 0
#     dataset.loc[dataset['DEF_30_CNT_SOCIAL_CIRCLE' + suffix].isnull(),'DEF_30_CNT_SOCIAL_CIRCLE' + suffix] = 0
#     dataset.loc[dataset['OBS_60_CNT_SOCIAL_CIRCLE' + suffix].isnull(),'OBS_60_CNT_SOCIAL_CIRCLE' + suffix] = 0
#     dataset.loc[dataset['DEF_60_CNT_SOCIAL_CIRCLE' + suffix].isnull(),'DEF_60_CNT_SOCIAL_CIRCLE' + suffix] = 0
    
#     dataset.loc[dataset['AMT_REQ_CREDIT_BUREAU_HOUR' + suffix].isnull(),'AMT_REQ_CREDIT_BUREAU_HOUR' + suffix] = 0
#     dataset.loc[dataset['AMT_REQ_CREDIT_BUREAU_DAY' + suffix].isnull(),'AMT_REQ_CREDIT_BUREAU_DAY' + suffix] = 0
#     dataset.loc[dataset['AMT_REQ_CREDIT_BUREAU_WEEK' + suffix].isnull(),'AMT_REQ_CREDIT_BUREAU_WEEK' + suffix] = 0
#     dataset.loc[dataset['AMT_REQ_CREDIT_BUREAU_MON' + suffix].isnull(),'AMT_REQ_CREDIT_BUREAU_MON' + suffix] = 0
#     dataset.loc[dataset['AMT_REQ_CREDIT_BUREAU_QRT' + suffix].isnull(),'AMT_REQ_CREDIT_BUREAU_QRT' + suffix] = 0
#     dataset.loc[dataset['AMT_REQ_CREDIT_BUREAU_YEAR' + suffix].isnull(),'AMT_REQ_CREDIT_BUREAU_YEAR' + suffix] = 0
    
    #商品价格-填充空值
    dataset.loc[dataset['AMT_GOODS_PRICE'] == 0,'AMT_GOODS_PRICE'] = dataset['AMT_GOODS_PRICE'].median()
    #每期还款-填充空值
    dataset.loc[dataset['AMT_ANNUITY'] == 0,'AMT_ANNUITY'] = dataset['AMT_ANNUITY'].median()
    #受教育程度从低到高分成5个等级
    dataset['NAME_EDUCATION_TYPE'] = dataset['NAME_EDUCATION_TYPE'].map({"Lower secondary":1,"Secondary / secondary special":2,"Incomplete higher":3,"Higher education":4,"Academic degree":5})
    
    combine_origin[i] = dataset
    
    # 收入/信贷金额
    dataset['RATE_INCOME_CREDIT'] = dataset['AMT_INCOME_TOTAL'] / dataset['AMT_CREDIT']
    # 收入/商品价格
    dataset['RATE_INCOME_GPRICE'] = dataset['AMT_INCOME_TOTAL'] / dataset['AMT_GOODS_PRICE']
    # 每期还款/信贷金额
    dataset['RATE_ANNUITY_CREDIT'] = dataset['AMT_ANNUITY'] / dataset['AMT_CREDIT']
    # 每期还款/收入
    dataset['RATE_ANNUITY_INCOME'] = dataset['AMT_ANNUITY'] / dataset['AMT_INCOME_TOTAL']
    
    df = pd.concat([df,dataset[['RATE_INCOME_CREDIT','RATE_INCOME_GPRICE','RATE_ANNUITY_CREDIT','NAME_EDUCATION_TYPE']]],axis=1)
    
    combine.append(df)


In [32]:
# combine_origin[0].head()

In [23]:
# 选择特性
# X_train = combine_origin[0].drop(["TARGET",'SK_ID_CURR'], axis=1)
# y_train = combine_origin[0]['TARGET']

# select = SelectPercentile(percentile=20)
# select.fit(X_train, y_train)

# X_train_selected = select.transform(X_train)


In [27]:
round(6750 / 10000,2)

0.68

(307511, 6)

In [106]:
df_train = combine[0]
df_test = combine[1]

X_train = df_train.drop(["TARGET",'SK_ID_CURR'], axis=1)
Y_train = df_train["TARGET"]

X_test  = df_test.drop("SK_ID_CURR", axis=1).copy()

In [45]:
# model = linear_model.LinearRegression()
# model.fit(X_train, Y_train)
# Y_pred = model.predict(X_test)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [107]:
logreg = LogisticRegression()
logreg.fit(X_train,Y_train)
# Y_pred = logreg.predict(X_test)
Y_pred = logreg.predict_proba(X_test)
# acc_log = logreg.score(X_train,Y_train)

D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [108]:
pred = []
for t in Y_pred:
    pred.append(t[1])

In [111]:
submission = pd.DataFrame({
        "SK_ID_CURR": df_test_origin["SK_ID_CURR"],
        "TARGET": pred
    })

In [112]:
submission.to_csv('result/submission.csv', index=False)